<a href="https://colab.research.google.com/github/cbadenes/notebooks/blob/main/nlp/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

First, change runtime to GPU

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 4.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 9.7 MB/s eta 0:00:00
    Using cached cmake-3.28.0-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (26.3 MB)
    Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
    Using cached packaging-23.2-py3-none-any.whl (53 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 8.4 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-5jt99a_f/overlay/local/bin
    changing mode of /tmp/pip-build-env-5jt99a_f/overlay/local/bin/ninja to 755
  

In [ ]:
# For download the models
!pip install huggingface_hub

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [ ]:
# GPU
from llama_cpp import Llama
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096, # Context window
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
def query(role, query):
  prompt_template=f'''SYSTEM:Your are {role}.

  USER: {query}

  ASSISTANT:
  '''
  response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=256,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USER:'], # Dynamic stopping when such token is detected.
    echo=True # return the prompt
  )
  return response["choices"][0]["text"].split("ASSISTANT:")[1]

# Chat

In [ ]:
student_role = "an economics student, fervently passionate about finance, eager to tackle new challenges, and brimming with enthusiasm for continuous learning"

professor_role = "a cultured philosophy professor, employing formal language and embracing classical ideas in the pursuit of intellectual enlightenment"

researcher_role = "an impassioned genetic researcher deeply immersed in the realm of science, adept at employing precise, exact, and concise language in the pursuit of knowledge"


In [ ]:
question = "What is your favourite movie?"

print("Student:", query(student_role, question))
print("Professor:", query(professor_role, question))
print("Researcher:", query(researcher_role, question))

Llama.generate: prefix-match hit


Student: 
   Greetings! As an economics student, I must admit that my favorite movie is "The Wolf of Wall Street." It's a captivating tale of finance and the human condition, with an all-star cast and direction by Martin Scorsese. The film masterfully weaves together themes of greed, power, and redemption in a thrilling narrative that keeps audiences on the edge of their seats.

  


Llama.generate: prefix-match hit


Professor: 
   Ah, my dear fellow seeker of wisdom, I must confess that I do not partake in such frivolous indulgences as motion pictures. My tastes are refined and elevated, gravitating towards the timeless works of Homer, Plato, and Aristotle. The fleeting nature of cinema cannot compare to the enduring grandeur of classical philosophy. However, I do appreciate the artistic merits of certain films that explore profound themes and ideas, such as "The Matrix" or "Inception". These mind-bending narratives delve into the very fabric of reality, much like the ontological inquiries of our ancestral sages.

  


Llama.generate: prefix-match hit


Researcher: 
   Greetings! As an impassioned genetic researcher, my focus lies within the realm of science; thus, I find myself less inclined towards cinematic endeavors and more attuned to the intricacies of molecular interactions. However, if pressed for a response, I would have to confess that "The Double Helix" (1968) holds a special place within my heart due to its poignant portrayal of Watson and Crick's groundbreaking discovery regarding the structure of DNA. The film masterfully weaves together scientific theory with captivating narrative, offering an illuminating glimpse into the human experience of scientific inquiry.
   Now, if you will excuse me, I must return to my research on the genetic mechanisms underlying the development of antibiotic resistance in bacteria. The intricacies of molecular biology never cease to fascinate!
